#### Normal CNN for the Bird classification

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
DIR_TRAIN = "\\Users\\Maxi\\Downloads\\archive\\train\\"
DIR_VALID = "\\Users\\Maxi\\Downloads\\archive\\valid\\"
DIR_TEST = "\\Users\\Maxi\\Downloads\\archive\\test\\"

In [4]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_datagen = train_datagen.flow_from_directory(DIR_TRAIN,
                                                target_size=(224,224), 
                                                color_mode='rgb', 
                                                class_mode='categorical', 
                                                batch_size=32,
                                                shuffle=True)

valid_datagen = valid_datagen.flow_from_directory(DIR_VALID,
                                                target_size=(224,224), 
                                                color_mode='rgb', 
                                                class_mode='categorical', 
                                                batch_size=32,
                                                shuffle=True)

test_datagen = test_datagen.flow_from_directory(DIR_TEST,
                                                target_size=(224,224), 
                                                color_mode='rgb', 
                                                class_mode='categorical', 
                                                batch_size=32,
                                                shuffle=True)

Found 56046 images belonging to 385 classes.
Found 1925 images belonging to 385 classes.
Found 1925 images belonging to 385 classes.


In [5]:
def callable_generator(generator):
    def gen():
        for x,y in generator:
            yield x,y
    return gen

train = callable_generator(train_datagen)
valid = callable_generator(valid_datagen)
test = callable_generator(test_datagen)

In [6]:
train_dataset = tf.data.Dataset.from_generator(train, output_types=(tf.float32, tf.float32), output_shapes=((32,224,224,3), (32,)))
valid_dataset = tf.data.Dataset.from_generator(valid, output_types=(tf.float32, tf.float32), output_shapes=((32,224,224,3), (32,)))
test_dataset = tf.data.Dataset.from_generator(test, output_types=(tf.float32, tf.float32), output_shapes=((32,224,224,3), (32,)))

In [7]:
train_dataset

<FlatMapDataset shapes: ((32, 224, 224, 3), (32,)), types: (tf.float32, tf.float32)>

In [8]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.layer = [
            tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=(4,4), strides=(4,4)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=(4,4), strides=(4,4)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(385, activation='softmax')
        ]

    def call(self, x):

        for i in self.layer:
            x = i(x)
        return x

In [9]:
net = ConvNet()

net.build(input_shape=(32, 224, 224, 3))
net.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
net.fit(train_datagen, epochs=5, validation_data=valid_datagen)

ImportError: Image transformations require SciPy. Install SciPy.